In [89]:
# !pip install pandas
# !pip install geodesic
# !pip install folium

In [90]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic

### Setup data

In [91]:
raw_data_locations = """
20.9204743,104.367553
21.041574907282516,104.30301435291766
20.91771173725199,104.42808877676725
"""
raw_data_boundaries = """
106.728736877441,10.7497472763062 106.730430603027,10.7375707626343 106.715934753418,10.7383689880371 106.715866088867,10.7443914413452 106.715866088867,10.7456140518189 106.717964172363,10.7458124160767 106.726318359375,10.749481201172 106.728736877441,10.7497472763062
"""
# df_data_locations.head()


In [99]:
arr_data_locations = raw_data_locations.replace('\n',' ').replace(',',' ')
arr_data_boundaries = raw_data_boundaries.replace('\n',' ').replace(',',' ')

arr_data_locations = np.reshape(arr_data_locations.split(), (-1, 2))
arr_data_boundaries = np.reshape(arr_data_boundaries.split(), (-1, 2))

# df_data_locations.head()
df_data_locations = pd.DataFrame({'lat':arr_data_locations[:, 0], 'lng': arr_data_locations[:, 1]})
df_data_boundaries = pd.DataFrame({ 'lat':arr_data_boundaries[:, 1],'lng': arr_data_boundaries[:, 0]})

In [100]:
df_data_locations

,lat,lng
0,20.9204743,104.367553
1,21.041574907282516,104.30301435291766
2,20.91771173725199,104.42808877676725


In [101]:
df_data_boundaries

,lat,lng
0,10.7497472763062,106.728736877441
1,10.7375707626343,106.730430603027
2,10.7383689880371,106.715934753418
3,10.7443914413452,106.715866088867
4,10.7456140518189,106.715866088867
5,10.7458124160767,106.717964172363
6,10.749481201172,106.726318359375
7,10.7497472763062,106.728736877441


In [102]:
def cal_distance_between_2_points(gdf, index1, index2):
    """
    cal_distance_between_2_points
    """
    coords_1 = (gdf.loc[index1,"lat"], gdf.loc[index1,"lng"])
    coords_2 = (gdf.loc[index2,"lat"],  gdf.loc[index2,"lng"])
    distance = geodesic(coords_1, coords_2).meters
    return distance


In [103]:
distances = []
for i in range(len(df_data_locations)):
    distance = 0
    for j in range(len(df_data_locations)):
        distance += cal_distance_between_2_points(df_data_locations,i,j)
    distances.append(distance)

df_data_locations['distances_total (m)'] = distances
df_data_locations['distances_avg (m)'] = df_data_locations['distances_total (m)']/(len(df_data_locations)-1)

df_data_locations

,lat,lng,distances_total (m),distances_avg (m)
0,20.9204743,104.367553,21298.332417,10649.166208
1,21.041574907282516,104.30301435291766,33893.782713,16946.891356
2,20.91771173725199,104.42808877676725,25204.925957,12602.462979


### Visualize on Map


In [104]:
import folium
from folium import Circle

In [105]:
str_popup = "{}, {}"
m = folium.Map(location=[df_data_locations.loc[0,"lat"], df_data_locations.loc[0,"lng"],], zoom_start=11)

# marker_cluster = folium.MarkerCluster().add_to(m)


# boundaries map
for i in range(0,len(df_data_boundaries)):
    Circle(
        location=[df_data_boundaries.iloc[i]['lat'], df_data_boundaries.iloc[i]['lng']],
        radius=20,
        color='green',
        popup=str_popup.format(df_data_boundaries.iloc[i]['lat'], df_data_boundaries.iloc[i]['lng'])).add_to(m)

# location map
for id,row in df_data_locations.iterrows():
    folium.Marker(location=[row['lat'],row['lng']],
                  popup=str_popup.format(row['lat'],row['lng'])).add_to(m)
m